In [1]:
import pandas as pd
from sqlalchemy import create_engine

Londing the rusher.csv

In [2]:
csv_file= "Resources/rusher.csv"
rusher_df=pd.read_csv(csv_file)
rusher_df

,rushId,playId,teamId,playerId,rushPosition,rushType,rushDirection,rushLandmark,rushYards,rushPrimary,rushTd,rushEnd,rushNull
0,15184506,15088563,2310,20130063,TE,rush,right,tackle,2,1,0.0,in bounds,0
1,15184496,15088531,4500,20140062,QB,kneel,NaN,NaN,-1,1,0.0,in bounds,0
2,15184510,15088575,4500,20140062,QB,qb scramble,right,end,3,1,0.0,ran out of bounds,0
3,15184471,15088462,2310,20140629,RB,rush,right,end,7,1,0.0,in bounds,0
4,15184477,15088479,2310,20140629,RB,rush,left,guard,2,1,0.0,in bounds,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
241156,2114453,2041094,2700,20050002,RB,rush,right,end,5,1,1.0,in bounds,0
241157,2114464,2041120,2700,20050002,RB,rush,NaN,NaN,0,1,0.0,in bounds,0
241158,2114466,2041123,2700,20050002,RB,rush,middle,center,3,1,0.0,in bounds,0
241159,2114467,2041124,2700,20050002,RB,rush,middle,center,-2,1,0.0,in bounds,0


In [3]:
#Dropping columns 
drop_rusher_df=rusher_df.drop(columns=['rushType', 'rushDirection', 'rushLandmark','rushEnd', 'rushNull','rushPrimary','playId'])

In [4]:
#Renaming columns 
new_rusher_df=drop_rusher_df.rename(columns={"rushId": 'rush_id', 'teamId': 'team_id','playerId':'player_id', 
                          'rushPosition': 'rush_position', 'rushYards': 'rush_yards','rushTd':'rush_td'}).head()


##Loading the reciver.csv 

In [5]:
rec_csv_file= "Resources/receiver.csv"
receiver_df=pd.read_csv(rec_csv_file)
receiver_df

,receiverId,playId,teamId,playerId,recPosition,recYards,rec,recYac,rec1down,recFumble,recPassDef,recPassInt,recEnd,recNull
0,15235157,15088495,4500,20100082,WR,18,1,0,0,0,0,0,in bounds,0
1,15235174,15088533,4500,20100082,WR,5,1,0,0,0,0,0,in bounds,0
2,15235175,15088534,4500,20100082,WR,15,1,2,0,0,0,0,in bounds,0
3,15235184,15088552,4500,20100082,WR,0,0,0,0,0,0,0,in bounds,0
4,15235213,15088610,4500,20100082,WR,0,0,0,0,0,0,0,in bounds,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306514,2266764,2041106,3900,20050430,WR,5,1,9,0,0,0,0,in bounds,0
306515,2266727,2041022,3900,20060025,WR,6,1,0,0,0,0,0,in bounds,0
306516,2266733,2041042,3900,20060025,WR,0,0,0,0,0,0,0,in bounds,0
306517,2266736,2041046,2700,20060082,WR,14,1,0,0,0,0,0,in bounds,0


In [6]:
#Selecting the columns 
new_receiver_df = receiver_df[['receiverId','teamId', 'playerId', 'recPosition', 'recYards']].copy()
new_receiver_df.head()

,receiverId,teamId,playerId,recPosition,recYards
0,15235157,4500,20100082,WR,18
1,15235174,4500,20100082,WR,5
2,15235175,4500,20100082,WR,15
3,15235184,4500,20100082,WR,0
4,15235213,4500,20100082,WR,0


In [7]:
name_receiver_df=new_receiver_df.rename(columns={'receiverId': 'receiver_id', 'teamId': 'team_id',
                                                'playerId': 'player_id', 'recPosition': 'rec_position',
                                               'recYards': 'rec_yards'})
name_receiver_df

,receiver_id,team_id,player_id,rec_position,rec_yards
0,15235157,4500,20100082,WR,18
1,15235174,4500,20100082,WR,5
2,15235175,4500,20100082,WR,15
3,15235184,4500,20100082,WR,0
4,15235213,4500,20100082,WR,0
...,...,...,...,...,...
306514,2266764,3900,20050430,WR,5
306515,2266727,3900,20060025,WR,6
306516,2266733,3900,20060025,WR,0
306517,2266736,2700,20060082,WR,14


##Conecting to the local data base

In [8]:
rds_connection_string = "postgres:postgres@localhost:5432/NFL_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [9]:
engine.table_names()

['draftnfl', 'playernfl', 'receiver', 'rusher']

## Loading the clean rusher.csv to database

In [18]:
new_rusher_df.to_sql(name='rusher', con=engine, if_exists='append', index=False)

In [19]:
pd.read_sql_query('select* from rusher', con=engine).head()

,rush_id,team_id,player_id,rush_position,rush_yards,rush_td
0,15184506,2310,20130063,TE,2.0,0
1,15184496,4500,20140062,QB,-1.0,0
2,15184510,4500,20140062,QB,3.0,0
3,15184471,2310,20140629,RB,7.0,0
4,15184477,2310,20140629,RB,2.0,0


## Loading the clean receiver.csv to database

In [20]:
name_receiver_df.to_sql(name='receiver', con=engine, if_exists='append', index=False)

In [21]:
pd.read_sql_query('select* from receiver', con=engine).head()

,receiver_id,team_id,player_id,rec_position,rec_yards
0,15235157,4500,20100082,WR,18
1,15235174,4500,20100082,WR,5
2,15235175,4500,20100082,WR,15
3,15235184,4500,20100082,WR,0
4,15235213,4500,20100082,WR,0


# Merging the receiver data

In [14]:
receiver_data=pd.read_sql_query('select playernfl.playerid, receiver.player_id, draftnfl.namefull, receiver.rec_yards, receiver.rec_position from playernfl inner join receiver on playernfl.playerId= receiver.player_id inner join draftnfl on playernfl.playerid= draftnfl.playerid'
, con=engine).head()
receiver_data

,playerid,player_id,namefull,rec_yards,rec_position
0,20100082,20100082,Emmanuel Sanders,18,WR
1,20100082,20100082,Emmanuel Sanders,5,WR
2,20100082,20100082,Emmanuel Sanders,15,WR
3,20100082,20100082,Emmanuel Sanders,0,WR
4,20100082,20100082,Emmanuel Sanders,0,WR


In [26]:
receiver_data_df=receiver_data.drop(columns=['playerid'])
receiver_data_df.head()

,player_id,namefull,rec_yards,rec_position
0,20100082,Emmanuel Sanders,18,WR
1,20100082,Emmanuel Sanders,5,WR
2,20100082,Emmanuel Sanders,15,WR
3,20100082,Emmanuel Sanders,0,WR
4,20100082,Emmanuel Sanders,0,WR


# # Merging the rush data

In [25]:
rush_data=pd.read_sql_query('select playernfl.playerid, rusher.player_id, draftnfl.namefull, rusher.rush_yards, rusher.rush_position from playernfl inner join rusher on playernfl.playerId= rusher.player_id inner join draftnfl on playernfl.playerid= draftnfl.playerid'
, con=engine)
rush_data.head()

,playerid,player_id,namefull,rush_yards,rush_position
0,20130063,20130063,Travis Kelce,2.0,TE
1,20130063,20130063,Travis Kelce,2.0,TE
2,20140062,20140062,Jimmy Garoppolo,3.0,QB
3,20140062,20140062,Jimmy Garoppolo,-1.0,QB
4,20140062,20140062,Jimmy Garoppolo,3.0,QB
5,20140062,20140062,Jimmy Garoppolo,-1.0,QB
